### CÁC THƯ VIỆN CẦN THIẾT

In [17]:
import requests
import re
import os
import csv
from underthesea import word_tokenize
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import clear_output

import pyodbc

### KẾT NỐI VÀ TẠO BẢNG DỮ LIỆU VÀO DATABASE DB_NEWS ĐÃ TẠO TRÊN SQL SERVER

In [19]:
### CONNECT TO CLOUD SQL SERVER USING PYODBC
SERVER = 'mssql-125320-0.cloudclusters.net'
PORT = 19147  # your port
UID = 'dang'
PASSWORD = 'Tranminhdang206'
DATABASE = 'db_news'

db_connect = pyodbc.connect(
  'DRIVER={ODBC Driver 17 for SQL Server};'
  'SERVER=%s,%s;'
  'DATABASE=%s;'
  'UID=%s;'
  'PWD=%s' % (SERVER, PORT, DATABASE, UID, PASSWORD))

### CREATE TABLE NEWS AND LABELS 
sql_scripts = """
CREATE TABLE labels (
    label_id CHAR(2) PRIMARY KEY,
    label NVARCHAR(MAX)
    );

CREATE TABLE news (
    new_id CHAR(7) PRIMARY KEY,
    url VARCHAR(MAX) NULL,
    label_id CHAR(2) REFERENCES labels(label_id),
    title NVARCHAR(MAX) NULL,
    content NVARCHAR(MAX) NULL
);
"""
cursor = db_connect.cursor()
cursor.execute(sql_scripts)
cursor.commit()

### IMPORT DATA VÀO BẢNG LABELS
sql_scripts = """
INSERT INTO labels (label_id, label)
VALUES
    ('01', N'Đời Sống'),
    ('02', N'Du Lịch'),
    ('03', N'Giải Trí'),
    ('04', N'Giáo Dục'),
    ('05', N'Khoa Học'),
    ('06', N'Kinh Doanh'),
    ('07', N'Pháp Luật'),
    ('08', N'Sức Khỏe'),
    ('09', N'Thể Thao');
"""
cursor = db_connect.cursor()
cursor.execute(sql_scripts)
cursor.commit()

In [23]:
### GET DATA TABLE LABELS
sql_scripts = """
select * from labels
"""
cursor = db_connect.cursor()
cursor.execute(sql_scripts)
data_label = cursor.fetchall()
df_label = pd.DataFrame.from_records(data_label, columns=[column[0] for column in cursor.description])
labels_map = {x['label']:x['label_id'] for x in df_label.to_dict('records')}
df_label

,label_id,label
0,01,Đời Sống
1,02,Du Lịch
2,03,Giải Trí
3,04,Giáo Dục
4,05,Khoa Học
5,06,Kinh Doanh
6,07,Pháp Luật
7,08,Sức Khỏe
8,09,Thể Thao


### TRÍCH XUẤT DỮ LIỆU TỪ CÁC FILE HTML ĐÃ CÀO VÀ IMPORT VÀO BẢNG NEWS VỪA TẠO

In [20]:
folders_data_html = os.listdir('data_html')
path_data_html = 'data_html'

table_name = 'news'
with open('data_news_raw.csv', mode='w') as csv_file:
	new_id = 0
	for i, folder_label in enumerate(folders_data_html):
		if not folder_label.endswith('.ini'):
			path_cur_folder = os.path.join(path_data_html, folder_label)
			list_files_html = os.listdir(path_cur_folder)
			fieldnames = ['new_id','url','label_id','title', 'content']
			writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
			# Ghi header của file CSV
			writer.writeheader()
			# Lặp qua tất cả các tệp tin trong thư mục và các thư mục con
			for idx, file_name in enumerate(list_files_html):
				print(f"File {idx +1}/{len(list_files_html)} of label [{folder_label}] - ({i + 1}/{len(folders_data_html)})")
				# Kiểm tra xem tệp tin có phải là tệp tin HTML hay không
				if file_name.endswith('.html'):
					# Mở tệp tin và đọc nội dung
					new_id += 1
					str_new_id = str(new_id).zfill(7)
					with open(os.path.join(path_cur_folder, file_name), 'r') as file:
						content = file.read()
						content_html = BeautifulSoup(content, 'html.parser')
						# Lấy url của bài báo
						url_new = content_html.select_one('[property="og:url"]').attrs['content']
						# Lấy title của văn bản
						title_detail = content_html.select_one('h1.title-detail')
						if title_detail:
							title = title_detail.text
						# Lấy title của video
						title_video = content_html.select_one('h1.title')
						if title_video:
							title = title_video.text
						summary =" "
						for p in content_html.find_all('p', {'class': 'Normal'}):
							summary += p.text.strip() + ' '
						# Ghi thông tin vào file CSV
						writer.writerow({'new_id':str_new_id,'url':url_new,'label_id':labels_map[folder_label],'title':title, 'content':summary})
						# Ghi thông tin vào bảng news
						cursor = db_connect.cursor()
						row_to_insert  = tuple([str_new_id,url_new,labels_map[folder_label],title,summary])
						query = f"INSERT INTO {table_name} VALUES ({','.join('?' * len(row_to_insert))})"
						cursor.execute(query, row_to_insert)
						cursor.commit()
						print(f"Imported {row_to_insert}")
						cursor.close()
				clear_output()

In [27]:
sql_scripts = """
select count(*) from news
"""
cursor = db_connect.cursor()
cursor.execute(sql_scripts)
print(f"Bảng news có số dòng là {cursor.fetchall()[0][0]}")

Bảng news có số dòng là 5570


In [28]:
sql_scripts = """
select top 10 * from news
"""
cursor = db_connect.cursor()
cursor.execute(sql_scripts)
print("10 dòng đầu")
cursor.fetchall()

10 dòng đầu


[('0000001', 'https://vnexpress.net/samsung-xac-dinh-viet-nam-la-cu-diem-san-xuat-toan-cau-4593216.html', '06', "Samsung xác định Việt Nam là 'cứ điểm sản xuất toàn cầu' ", ' Nhận định này được ông Park Hark Kyu, Tổng giám đốc phụ trách tài chính của Tập đoàn Samsung, nêu khi gặp Thủ tướng Phạm Minh Chính chiều nay. Ông Park đánh giá cao môi trường đầu tư, kinh doanh và xác định Việt Nam là cứ điểm sản xuất toàn cầu. Hãng cũng hướng tới mục tiêu đưa Việt Nam thành "trung tâm của các trung tâm" nghiên cứu, phát triển. Cuối năm ngoái, Trung tâm R&D của Samsung tại Hà Nội đã vận hành, đưa ra giải pháp đào tạo nhân lực để có nhiều hơn nữa người Việt tham gia đội ngũ lãnh đạo của hãng ở Việt Nam. Hiện có hơn 2.000 kỹ sư đang làm việc tại trung tâm này, góp phần phát triển công nghiệp, nâng cao tỷ lệ nội địa hóa, chuỗi cung ứng sản xuất. Số nhà cung cấp cấp 1 và 2 của Việt Nam trong chuỗi cung ứng toàn cầu của Samsung đã tăng 10 lần, từ 25 doanh nghiệp vào 2014 lên 257 vào cuối năm 2022. Ngo

### HÀM XỬ LÍ LOẠI BỎ SỐ VÀ KÍ TỰ ĐẶC BIỆT TRONG CÁC TRƯỜNG VĂN BẢN

In [30]:
sql_scripts = """
create FUNCTION UpdateVietnameseText (@input nvarchar(max))
RETURNS nvarchar(max)
AS
BEGIN
  DECLARE @output nvarchar(max)
  SET @output = LOWER(@input);
  WHILE PATINDEX('%[0-9]%', @output) > 0
    BEGIN 
        SET @output = STUFF(@output, PATINDEX('%[0-9]%', @output), 1, '')
    END
  WHILE PATINDEX('%[^a-zA-Z ]%', @output) > 0
    BEGIN
        SET @output = STUFF(@output, PATINDEX('%[^a-zA-Z ]%', @output), 1, '')
    END
  SET @output = LTRIM(RTRIM(@output));
  WHILE PATINDEX('%  %', @output) > 0
    BEGIN
        SET @output = STUFF(@output, PATINDEX('%  %', @output), 1, '')
    END
  RETURN @output
END
"""
cursor = db_connect.cursor()
cursor.execute(sql_scripts)
cursor.commit()

In [32]:
### TIỀN XỬ LÍ CỘT TITLE VÀ CONTENT TRONG BẢNG NEWS
sql_scripts = """
UPDATE news
SET title = dbo.UpdateVietnameseText(title)
"""
cursor = db_connect.cursor()
cursor.execute(sql_scripts)
cursor.commit()

sql_scripts = """
UPDATE news
SET content = dbo.UpdateVietnameseText(content)
"""
cursor = db_connect.cursor()
cursor.execute(sql_scripts)
cursor.commit()
# cursor.fetchall()

In [33]:
sql_scripts = """
select top 10 * from news
"""
cursor = db_connect.cursor()
cursor.execute(sql_scripts)
print("10 dòng đầu")
cursor.fetchall()

10 dòng đầu


[('0000001', 'https://vnexpress.net/samsung-xac-dinh-viet-nam-la-cu-diem-san-xuat-toan-cau-4593216.html', '06', 'samsung xác định việt nam là cứ điểm sản xuất toàn cầu', 'nhận định này được ông park hark kyu tổng giám đốc phụ trách tài chính của tập đoàn samsung nêu khi gặp thủ tướng phạm minh chính chiều nay ông park đánh giá cao môi trường đầu tư kinh doanh và xác định việt nam là cứ điểm sản xuất toàn cầu hãng cũng hướng tới mục tiêu đưa việt nam thành trung tâm của các trung tâm nghiên cứu phát triển cuối năm ngoái trung tâm rd của samsung tại hà nội đã vận hành đưa ra giải pháp đào tạo nhân lực để có nhiều hơn nữa người việt tham gia đội ngũ lãnh đạo của hãng ở việt nam hiện có hơn kỹ sư đang làm việc tại trung tâm này góp phần phát triển công nghiệp nâng cao tỷ lệ nội địa hóa chuỗi cung ứng sản xuất số nhà cung cấp cấp và của việt nam trong chuỗi cung ứng toàn cầu của samsung đã tăng lần từ doanh nghiệp vào lên vào cuối năm ngoài ra ông park hark kyu đề xuất với thủ tướng một số 